# 2D Convolution 

```
tf.nn.conv2d(
    input,
    filter,
    strides,
    padding,
    use_cudnn_on_gpu=True,
    data_format='NHWC',
    dilations=[1, 1, 1, 1],
    name=None )
    
    
Input: Batch size (N) x Height (H) x Width (W) x Channels (C)

Filter: Height x Width x Input Channels x Output Channels
(e.g. [5, 5, 3, 64])
Strides: 4 element 1-D tensor, strides in each direction
(often [1, 1, 1, 1] or [1, 2, 2, 1])

Padding: 'SAME' or 'VALID'
Dilations: The dilation factor. If set to k > 1, there will be k-1 skipped cells between each filter element on that dimension.
Data_format: default to NHWC

```








## Convolutional layer

## A common practice is to group convolutional layer and non-linearity together, which we will do in this case

the input volume size (W)

the receptive field size of filter (F)

the stride with which they are applied (S)

the amount of zero padding used (P) on the border. 


## (W-F + 2P) / S + 1

In [0]:
def conv_relu(inputs , filters ,k_size1 ,k_size2, stride , padding , scope_name) :
  with tf.variable_scope(scope_name , reuse=tf.AUTO_RESUE) as scope :
    in_channels = inputs.shape[-1]
    kernel = tf.get_variable("kernel" , [k_size_1, k_size_2, in_channels , filters],
                            initializer = tf.truncated_normal_initializer())
    biases =tf.get_variable("biases" ,[filters] , initializer=tf.random_normal_initializer())
    conv = tf.nn.conv2d(inputs, kernel , strides=[1,stride, stride , 1] ,padding=padding)
    return tf.nn.relu(conv+bias ,name= scope_name)
  

# Pooling
##  Pooling is a downsampling technique to reduce the dimensionality of the feature map extracted from the convolutional layer in order to reduce the processing time.

We can compute the spatial size on each dimension (width/depth) of the output of pooling layer a function of:

the input volume size (W)

the pooling size (K)

the stride with which they are applied (S)

the amount of zero padding used (P) on the border. 

The formula is as followed:

## (W-K + 2P) / S + 1


In [0]:
def maxpool(inputs, ksize, stride, padding = "VALID" , scope_name="pool") :
  with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope :
    pool = tf.nn.max_pool(inputs , ksize= [1,ksize, ksize ,1], 
                         strides = [1,stride, stride, 1],
                         padding=padding)
    return pool
  

# Fully connected

### Fully connected, or dense, layer is called so because every node in the layer is connected to every node in the preceding layer.

### Convolutional layers are only locally connected.


In [0]:
def fully_connected(inputs, out_dim , scope_name="fc") :
  with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope :
    in_dim = inputs.shape[-1]
    w= tf.get_variable("weights" ,[in_dim ,out_dim] , 
                      initializer=tf.truncated_normal_initializer())
    b = tf.get_variable("biases" ,[out_dim] , initializer = tf.constant_initializer(0.0))
    
    out=tf.matmul(inputs,w)+b
    return out

In [0]:
def inference(self):
        conv1 = conv_relu(inputs=self.img,
                        filters=32,
                        k_size=5,
                        stride=1,
                        padding='SAME',
                        scope_name='conv1')
        pool1 = maxpool(conv1, 2, 2, 'VALID', 'pool1')  
        conv2 = conv_relu(inputs=pool1,                     # reuse=tf.AUTO_REUSE 효과 굳굳
                        filters=64,
                        k_size=5,
                        stride=1,
                        padding='SAME',
                        scope_name='conv2')
        pool2 = maxpool(conv2, 2, 2, 'VALID', 'pool2')  # reuse=tf.AUTO_REUSE
        feature_dim = pool2.shape[1] * pool2.shape[2] * pool2.shape[3]
        pool2 = tf.reshape(pool2, [-1, feature_dim])
        fc = tf.nn.relu(fully_connected(pool2, 1024, 'fc'))
        dropout = tf.layers.dropout(fc, self.keep_prob, training=self.training, name='dropout')
        
        self.logits = fully_connected(dropout, self.n_classes, 'logits')
                                                         
  
  

"VALID" only ever drops the right-most columns (or bottom-most rows).

"SAME" tries to pad evenly left and right, but if the amount of columns to be added is odd, it will add the extra column to the right, as is the case in this example (the same logic applies vertically: there may be an extra row of zeros at the bottom).

In [0]:
def eval(self) :
        '''
        Count the number of right predictions in a batch
        '''
    with tf.name_scope("predict") :
      preds =tf.nn.softmax(self.logits)
      correct_preds =tf.equal(tf.argmax(preds,1),tf.argmax(self.label,1))
      self.accuracy= tf.reduce_sum(tf.cast(correct_preds, ft.float32))
      

In [0]:
import numpy as np
a= np.random.standard_normal(size=[1000]).reshape(10,10,2,5)
print(a.shape)
z = a.shape[1]*a.shape[2]*a.shape[3]
k=np.reshape(a, [-1, z])


#   tf.layers
## I have  a confession to make: we’ve been learning how to do it the hard way.
## We actually don’t have to write the conv_relu method, pooling, or fully_connected.
##  tf.layers that provides many off-the-shelf layers. Higher level libraries like Keras, Sonnet also provide ready-made models that you can call with a few lines  of code. 

```
conv1 = tf.layers.conv2d(inputs=self.img,
                                  filters=32,
                                  kernel_size=[5, 5],
                                  padding='SAME',
                                  activation=tf.nn.relu,
                                  name='conv1')

pool1 = tf.layers.max_pooling2d(inputs=conv1, 
                                        pool_size=[2, 2], 
                                        strides=2,
                                        name='pool1')

fc = tf.layers.dense(pool2, 1024, activation=tf.nn.relu, name='fc')

dropout = tf.layers.dropout(fc, 
                                    self.keep_prob, 
                                    training=self.training, 
                                    name='dropout')
                                  
```
It’s pretty straightforward to use tf.layers. 

There’s only one tiny thing to note. With tf.layers.dropout, you need to have another variable to indicate whether it’s in the training mode or in the evaluation mode. 

We want to drop out neurons during training, but we want to use all of them during evaluating!

Drop neurals during training
Want to use all of them during testing


